In [1]:
import pandas as pd
import numpy as np
import io, re
from unidecode import unidecode
from gensim.models import KeyedVectors
from nltk.tokenize import word_tokenize

In [2]:
archive = "Corpus.csv"

In [3]:
with open(archive, 'r') as myfile:
    data = myfile.read().encode('latin1').decode('utf8')
    df = pd.read_csv(io.StringIO(re.sub('"\s*\n','"',data)), delimiter=',')

In [4]:
df.head(10)

,#,Frases,Tipo,Resposta,Ação a ser Realizada,Ações Necessárias
0,1,Ligar luz,Explicito,Acendendo a lâmpada.,Acender a Lâmpada,Acender a Lâmpada
1,2,Ligar a luz,Explicito,Acendendo a lâmpada.,Acender a Lâmpada,Apagar Lâmpada
2,3,Iluminar o ambiente,Implicito,Deseja acender a lâmpada?,Acender a Lâmpada,Ligar Ventilador
3,4,Já está claro o dia!,Implicito,Apagando a lâmpada.,Apagar Lâmpada,Desligar Ventilador
4,5,Vamos economizar energia?,Implicito,Deseja apagar a lâmpada?,Apagar Lâmpada,Ligar Televisão
5,6,"Casa, apague a luz",Explicito,Apagando a lâmpada.,Apagar Lâmpada,Desligar Televisão
6,7,O ar está muito parado,Implicito,Ligando o ventilador.,Ligar Ventilador,Aumentar Volume
7,8,Está muito quente aqui,Implicito,Ligando o ventilador.,Ligar Ventilador,Diminuir Volume
8,9,Que calor!,Implicito,Deseja ligar o ventilador?,Ligar Ventilador,Mudar Canal
9,10,"Está muito barulho, desligue o ventilador",Explicito,Desligando o ventilador.,Desligar Ventilador,NaN


In [5]:
df = df.drop(columns=['#', 'Ações Necessárias'])

In [6]:
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [7]:
df['Frases'] = df['Frases'].apply(unidecode)

In [8]:
def replace_i(v):
    return v.replace('í', 'i')

In [9]:
df['Tipo'] = df['Tipo'].apply(replace_i)

In [10]:
df.head(5)

,Frases,Tipo,Resposta,Ação a ser Realizada
0,Ligar luz,Explicito,Acendendo a lâmpada.,Acender a Lâmpada
1,Ligar a luz,Explicito,Acendendo a lâmpada.,Acender a Lâmpada
2,Iluminar o ambiente,Implicito,Deseja acender a lâmpada?,Acender a Lâmpada
3,Ja esta claro o dia!,Implicito,Apagando a lâmpada.,Apagar Lâmpada
4,Vamos economizar energia?,Implicito,Deseja apagar a lâmpada?,Apagar Lâmpada


In [11]:
X = df['Frases'].tolist()
Y1 = df['Ação a ser Realizada'].tolist()
Y2 = df['Resposta'].tolist()
Y3 = df['Tipo'].tolist()

In [12]:
#Testando somente frases implícitas.
filteredExplicit = df[df.Tipo == 'Explicito']
filteredImplicit = df[df.Tipo == 'Implicito']
X_imp = filteredImplicit['Frases'].tolist()
X_exp = filteredExplicit['Frases'].tolist()
Y1_imp = filteredImplicit['Ação a ser Realizada'].tolist()
Y1_exp = filteredExplicit['Ação a ser Realizada'].tolist()
Y2_imp = filteredImplicit['Resposta'].tolist()
Y2_exp = filteredExplicit['Resposta'].tolist()

In [13]:
# Todas as frases
X = df['Frases'].tolist()
Y1 = df['Ação a ser Realizada'].tolist()
Y2 = df['Resposta'].tolist()
Y3 = df['Tipo'].tolist()

In [14]:
len(set(Y1)), len(set(Y2)), len(set(Y3))

(9, 18, 2)

# preprocessing data

In [15]:
# !pip install -U scikit-learn

# !pip install gensim

# !pip install nltk

In [16]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(Y1_imp)

Y1_encoded = le.transform(Y1)

from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

X_tfidf = vectorizer.fit_transform(X)

# using word embeddings

In [17]:
w2v = KeyedVectors.load_word2vec_format('glove_s100.txt')

In [23]:
def transform_w2v(X):
    X_w2v = []
    for sentence in X:
        tokens = word_tokenize(sentence)
        vectors = []
        for token in tokens:
            try:
                vectors.append(w2v.get_vector(token.lower()))
            except:
                print(token)
            
        X_w2v.append(np.mean(vectors, axis=0))
    return X_w2v

X_w2v = transform_w2v(X)

escuridao
ventilacao
friiiiio


In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer 
vectorizer = TfidfVectorizer()
print("TF-IDF:")
print(vectorizer.fit_transform(["Tá tudo muito alto aqui."]))
print("Word2Vec:")
print(transform_w2v(["Tá tudo muito alto aqui."]))

TF-IDF:
  (0, 1)	0.4472135954999579
  (0, 0)	0.4472135954999579
  (0, 2)	0.4472135954999579
  (0, 3)	0.4472135954999579
  (0, 4)	0.4472135954999579
Word2Vec:
[array([-3.05735856e-01, -1.77687511e-01, -3.31434488e-01, -8.48858356e-01,
       -2.96531498e-01,  2.51416373e-03, -2.96251982e-01, -3.19136649e-01,
       -2.19754994e-01, -1.29450485e-01,  5.66140153e-02,  3.39437336e-01,
        4.83064950e-02, -5.40972531e-01,  9.80334878e-02,  1.57491505e-01,
        2.48661831e-01,  2.47282162e-01,  3.02413344e-01,  8.17823946e-01,
        5.98798990e-01,  5.45873344e-02,  3.70278507e-01, -4.15192366e-01,
       -5.40721655e-01, -6.60332978e-01,  9.96828079e-04, -5.88498473e-01,
        3.47100645e-01,  1.65343249e+00,  7.23456666e-02,  3.34975809e-01,
       -1.12200655e-01,  2.26949170e-01,  2.55855173e-01, -4.32572335e-01,
        8.94918367e-02, -2.05582842e-01, -5.73872864e-01,  4.53648001e-01,
        1.19539507e-01, -1.24653161e-01, -5.87590002e-02, -3.10654491e-01,
        8.445683

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_w2v, Y1_encoded, test_size=0.25, random_state=42, stratify=Y1_encoded)

# Experiments with classification algorithms

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import plot_confusion_matrix
from sklearn.model_selection import GridSearchCV

In [ ]:
import warnings
warnings.filterwarnings('ignore')
params_lr = {'penalty': ['l1', 'l2'], 'C': [0.1, 0.5, 1, 10, 50], 'fit_intercept': [True, False],
             'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']}

params_svc = {'C': [0.1, 0.5, 1, 10, 20], 'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
             'shrinking': [True, False]}

params_knn = {'n_neighbors': [9, 11, 13, 15], 'weights':['uniform', 'distance'],
             'algorithm': ['auto', 'ball_tree', 'kd_tree'], 'leaf_size': [15, 30, 45]}

params_tree = {'criterion': ['gini', 'entropy'], 'splitter':['best', 'random'],
               'min_samples_split':[1, 2, 4, 6], 'max_features': ['auto', 'sqrt', 'log2']}

params_clf = {'learning_rate': [0.05, 0.1, 0.25, 0.5],
              'n_estimators': [50, 100, 200], 'criterion':['friedman_mse', 'mse'],
              'min_samples_split': [1, 2, 4], 'max_depth':[2,3,5,7], 'max_features':['auto', 'sqrt', 'log2']}

lr = LogisticRegression(random_state=32)
svm = SVC(random_state=32)
knn = KNeighborsClassifier(n_jobs=-1)
tree = tree.DecisionTreeClassifier(random_state=32)
clf = GradientBoostingClassifier(random_state=32)

gs_lr = GridSearchCV(lr, params_lr, n_jobs=-1)
gs_lr.fit(X_train, y_train)
gs_svm = GridSearchCV(svm, params_svc, n_jobs=-1)
gs_svm.fit(X_train, y_train)
gs_knn = GridSearchCV(knn, params_knn, n_jobs=-1)
gs_knn.fit(X_train, y_train)
gs_tree = GridSearchCV(tree, params_tree, n_jobs=-1)
gs_tree.fit(X_train, y_train)
gs_clf = GridSearchCV(clf, params_clf, n_jobs=-1)
gs_clf.fit(X_train, y_train)

In [ ]:
#Anotar Hiperparâmetros para cada uma das Execuções.
print(gs_lr.best_params_)
print(gs_svm.best_params_)
print(gs_knn.best_params_)
print(gs_tree.best_params_)
print(gs_clf.best_params_ )

In [ ]:
lr = gs_lr.best_estimator_
svm = gs_svm.best_estimator_ 
knn9 = gs_knn.best_estimator_ 
tree = gs_tree.best_estimator_ 
clf = gs_clf.best_estimator_

y_pred_lr = lr.predict(X_test)
y_pred_svm = svm.predict(X_test)
y_pred_knn9 = knn9.predict(X_test)
y_pred_tree = tree.predict(X_test)
y_pred_grad = clf.predict(X_test)

In [ ]:
print(le.inverse_transform([0, 1, 2, 3, 4, 5, 6, 7, 8]))
plot_confusion_matrix(lr, X_test, y_test, normalize='true')
plot_confusion_matrix(svm, X_test, y_test, normalize='true')
plot_confusion_matrix(knn9, X_test, y_test, normalize='true')
plot_confusion_matrix(tree, X_test, y_test, normalize='true')
plot_confusion_matrix(clf, X_test, y_test, normalize='true')

In [ ]:
scores_lr = cross_val_score(lr, X_train, y_train, cv=5)
scores_svm = cross_val_score(svm, X_train, y_train, cv=5)
scores_knn9 = cross_val_score(knn9, X_train, y_train, cv=5)
scores_tree = cross_val_score(tree, X_train, y_train, cv=5)
scores_clf = cross_val_score(clf, X_train, y_train, cv=5)

In [ ]:
print("LR %0.2f accuracy with a standard deviation of %0.2f" % (scores_lr.mean(), scores_lr.std()))
print("SVM %0.2f accuracy with a standard deviation of %0.2f" % (scores_svm.mean(), scores_svm.std()))
print("KNN %0.2f accuracy with a standard deviation of %0.2f using %d neighbors" % (scores_knn9.mean(), scores_knn9.std(), 9))
print("Tree %0.2f accuracy with a standard deviation of %0.2f " % (scores_tree.mean(), scores_tree.std()))
print("Gradient Boosting %0.2f accuracy with a standard deviation of %0.2f " % (scores_clf.mean(), scores_clf.std()))

In [ ]:
from sklearn.metrics import classification_report

for i, c in enumerate(list(le.classes_)):
    print(i, c)

In [ ]:
print(classification_report(y_test, y_pred_lr))

In [ ]:
print(classification_report(y_test, y_pred_svm))

In [ ]:
print(classification_report(y_test, y_pred_knn9))

In [ ]:
print(classification_report(y_test, y_pred_tree))

In [ ]:
print(classification_report(y_test, y_pred_grad))

In [ ]:
# !pip install mlxtend

In [ ]:
import warnings
warnings.filterwarnings('ignore')

from mlxtend.evaluate import paired_ttest_5x2cv
methods = []
methods.append(["Logistic Regression", lr])
methods.append(["SVM", svm])
methods.append(["KNN", knn9])
methods.append(["Decision Trees", tree])
methods.append(["Gradient Boosting", clf])

for i in range(0, len(methods)):
    for j in range(i+1, len(methods)):
        t, p = paired_ttest_5x2cv(estimator1 = methods[i][1],
                                  estimator2 = methods[j][1],
                                  X=X_w2v, y=Y1,
                                  random_seed=1)
        print(str(methods[i][0]) + " x " + str(methods[j][0]))
        print('t statistic: %.3f' % t)
        print('p value: %.3f' % p)
        print()